In [1]:
!pip install gref4hsi==0.2.4 pykrige

# Install py6s using conda (assuming mamba is a conda alias)
try:
    import py6s
except ImportError:
    !mamba install -y py6s

!pip install rad4sea==0.0.1


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.1.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['py6s']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━━━━━━━━━╸━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB

In [2]:


# Standard python library
import configparser
import sys
import os
import argparse
from collections import namedtuple

# Local resources
from gref4hsi.scripts import georeference
from gref4hsi.scripts import orthorectification
from gref4hsi.scripts import coregistration
from gref4hsi.utils import parsing_utils, specim_parsing_utils
from gref4hsi.utils import visualize
from gref4hsi.utils.config_utils import prepend_data_dir_to_relative_paths
from gref4hsi.utils.config_utils import customize_config

# Third party
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# From seabeepy/notebooks/flight_runner
import datetime as dt
import os
from pathlib import Path

from config import SETTINGS
from pyodm import Node
from subprocess import CalledProcessError

import seabeepy as sb


In [4]:


# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)



In [5]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/ntnu",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs"

In [6]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2024-05-16 11:46:49.597365


In [7]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'capture' subdirectory, but NOT a 'processed' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and not sb.ortho.check_subdir_exists(f.parent, "processed")
]

# TODO: add criterion ""

In [8]:
mission_list

[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_seaside-straumen_202402191358_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_hopavaagen_202402191311_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_hopavaagen_202402191253_ntnu_hyperspectral_74m')]

In [9]:
# Establish the ancillary data paths, copy to local work space and 
from pathlib import Path
import shutil

geoid_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/geoids/no_kv_HREF2018A_NN2000_EUREF89.tif')
config_template_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini')
lab_calibration_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')

# Local path geoid
geoid_path = os.path.join(temp_dir, "no_kv_HREF2018A_NN2000_EUREF89.tif")
try:
    shutil.copyfile(geoid_path_minio, geoid_path)
except FileExistsError:
    pass

# Local path for configuration file
config_template_path = os.path.join(temp_dir, "config_template_path_specim.ini")
try:
    shutil.copyfile(config_template_path_minio, config_template_path)
except FileExistsError:
    pass

# Local path for lab calibration
lab_calibration_path = os.path.join(temp_dir, "lab-calibration")
try:
    shutil.copytree(lab_calibration_path_minio, lab_calibration_path)
except FileExistsError:
    pass

In [10]:
import process_hyperspectral

# Process missions in the list
for specim_mission_folder_minio in mission_list:

    mission_name = specim_mission_folder_minio.name
    print(f"\n################\nProcessing: {mission_name}")

    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)

    # Copy to local fs if not there already
    try:
        print('Copying file')
        shutil.copytree(specim_mission_folder_minio, specim_mission_folder)
    except FileExistsError:
        pass

    # The config file is read
    config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")

    # Start the specim processing (if it does not finish
    process_hyperspectral.main(str(config_yaml), 
                        str(specim_mission_folder), 
                        geoid_path, 
                        config_template_path, 
                        lab_calibration_path,
                        fast_mode = False)




################
Processing: slettvik_seaside-straumen_202402191358_ntnu_hyperspectral_74m
Copying file
DEM folder does not exist so Geoid is used as terrain instead
Coregistration is not done, as there was no reference orthomosaic


/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


*.xyz already exists, ignoring re-creation


Computing 2D Triangulation: 100%|██████████[00:00<00:00]


################ Georeferencing: ################
Georeferencing file 1/6, progress is 0.0 %


All rays were successfully intersected with trimesh
Mesh translation is [2804905.02667452  470455.75463252 5689827.80232584]
Georeferencing file 2/6, progress is 16.666666666666668 %
All rays were successfully intersected with trimesh
Mesh translation is [2804905.02667452  470455.75463252 5689827.80232584]
Georeferencing file 3/6, progress is 33.333333333333336 %
All rays were successfully intersected with trimesh
Mesh translation is [2804905.02667452  470455.75463252 5689827.80232584]
Georeferencing file 4/6, progress is 50.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2804905.02667452  470455.75463252 5689827.80232584]
Georeferencing file 5/6, progress is 66.66666666666667 %
All rays were successfully intersected with trimesh
Mesh translation is [2804905.02667452  470455.75463252 5689827.80232584]
Georeferencing file 6/6, progress is 83.33333333333333 %
All rays were successfully intersected with trimesh
Mesh translation is [2804905.02667452  470455.754

/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
Computing 2D Triangulation: 100%|██████████[00:00<00:00]



################ Georeferencing: ################
Georeferencing file 1/5, progress is 0.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2804746.98665109  471003.36093417 5689860.18608784]
Georeferencing file 2/5, progress is 20.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2804746.98665109  471003.36093417 5689860.18608784]
Georeferencing file 3/5, progress is 40.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2804746.98665109  471003.36093417 5689860.18608784]
Georeferencing file 4/5, progress is 60.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2804746.98665109  471003.36093417 5689860.18608784]
Georeferencing file 5/5, progress is 80.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2804746.98665109  471003.36093417 5689860.18608784]
[Open3D WARNING] Write Ply clamped color value to valid range

################ Orthorectifying: ########

KeyError: 'mission_epsg'

## 3. Convert radiance data into reflectance by division with simulated spectrum from Py6S

In [ ]:
# Raster and ancillary (
import rasterio
from rasterio.plot import show
import spectral as sp
import shutil

import glob

module_path = os.path.join('/home/notebook/', 'gitprojects/rad4sea/')
if module_path not in sys.path:
    sys.path.append(module_path)
import rad4sea
import rad2refl

# The radiance multiplier for the specim afx10 relative to W/(m^2*sr*nm)
radiance_multiplier = (1 / 1000) #(mW/cm^2*sr*um)*1000.0000 ->(mW/cm^2*sr*um)
radiance_multiplier *= (1e-3 / 1e-4) #(mW/cm^2*sr*um) -> (W/m^2*sr*um)
radiance_multiplier *= (1 / 1e3) # (W/m^2*sr*um) -> (W/m^2*sr*nm)



for specim_mission_folder_minio in mission_list:

    # Select a particular transect datacube:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)

    print(specim_mission_folder)

    # Where the cube and anc data are
    cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
    anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")

    # Make copies of radiance data for manipulation
    for filename in os.listdir(cube_folder):
        if filename.lower().endswith(".img") or filename.lower().endswith(".hdr"):  # Check for lowercase extension
            base, ext = os.path.splitext(filename)  # Separate base name and extension

            if base.split('_')[-1] == 'reflectance':
                pass
            else:
                new_filename = f"{base}_reflectance{ext}"  # Construct new filename with suffix
                source_file = os.path.join(cube_folder, filename)
                destination_file = os.path.join(cube_folder, new_filename)
                # Copy the file
                shutil.copyfile(source_file, destination_file)

    # We only modulate the copy of the data (with _reflectance suffix)
    cube_hdr_list = glob.glob(cube_folder + '/*_reflectance.hdr')

    # Simulates downwelling irradiance and computes remote sensing reflectance 
    rad2refl.main(anc_folder = anc_folder, cube_list_refl = cube_hdr_list, cube_folder = cube_folder, radiance_multiplier=radiance_multiplier)



In [ ]:
"""A simple demonstration of the data for a transect showing radiance, reflectance and an RGB composite"""

# See if there is one
specim_mission_folder_minio = mission_list[0]
specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")

cube_hdr_list = glob.glob(cube_folder + '/*_reflectance.hdr')
import matplotlib.pyplot as plt

# Example plotting of radiance data
refl_image_obj = sp.io.envi.open(cube_hdr_list[0])


band_im_refl = refl_image_obj[:, :, 100]
nodata = float(refl_image_obj.metadata['data ignore value'])
wl = np.array(refl_image_obj.metadata["wavelengths"]).astype(np.float64)

valids = (band_im_refl != nodata).squeeze()

n_rows, n_cols, _ = band_im_refl.shape

row_indices = np.repeat(np.arange(n_rows).reshape((-1, 1)), n_cols, axis = 1)
col_indices = np.repeat(np.arange(n_cols).reshape((1, -1)), n_rows, axis = 0)
#print(row_indices.shape)
row_valids = row_indices[valids]
col_valids = col_indices[valids]

n_valids = row_valids.size
spec_idx_example = int(n_valids/2) # Some random spectrum in the image. Index can be from 0 to n_valids


print(cube_hdr_list[0])
plt.plot(wl, refl_image_obj[row_valids[spec_idx_example], col_valids[spec_idx_example], :].flatten())
plt.ylabel('Remote sensing reflectance [1/sr]')
plt.xlabel('Nanometers [nm]')
plt.show()

rad_image_obj = sp.io.envi.open('_'.join(cube_hdr_list[0].split('_')[0:-1]) + '.hdr') 
rad_specim_example = rad_image_obj[row_valids[spec_idx_example], col_valids[spec_idx_example], :].flatten()
plt.plot(wl, rad_specim_example*radiance_multiplier)
plt.ylabel('Radiance [mW/(m$^2$ sr nm)]')
plt.xlabel('Nanometers [nm]')
plt.show()

pad = 10
rows_zoom = np.arange(row_valids[spec_idx_example]-pad, row_valids[spec_idx_example]+pad)
cols_zoom = np.arange(col_valids[spec_idx_example]-pad,col_valids[spec_idx_example]+pad)



RGB = refl_image_obj[:, :, [73,50,24]]

RGB.shape

# Emulates a gamma stretch to bring out contrast in dark areas
plt.imshow((RGB/RGB.max())**0.44, vmin = 0)
plt.scatter(row_valids[spec_idx_example], col_valids[spec_idx_example], label = 'Sample spectrum')
plt.legend()
plt.show()




## 4. Transfer to MinIO and Publish to GeoNode

### 4.1 Convert RGB composites into one one mosaic (no fancy stretching or similar yet)


In [ ]:
from osgeo import gdal
import rasterio as rio

def merge_rasters(raster_file_list, output_file):
    """Merges the geotif files in a directory into one geoTIF"""
    ds_lst = list()
    for raster in raster_file_list:
        ds = gdal.Warp('', raster, format='vrt')
        ds_lst.append(ds)
    dataset = gdal.BuildVRT('', ds_lst)
    ds1 = gdal.Translate(output_file, dataset)
    del ds1  
    del dataset
    # Add band info if not there yet
    with rio.open(output_file, "r+") as src:
        if any(val is None for val in src.descriptions):
            src.descriptions = ("red", "green", "blue")
    
    return output_file

### 4.2 Copy processed data to specim_mission_folder_minio/processed

In [ ]:

for specim_mission_folder_minio in mission_list:
    # Select a particular transect datacube:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    
    # Where the cube and anc data are
    cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
    anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")
    
    # Processed composites are now stored under
    composites_path = os.path.join(specim_mission_folder, 'processed/Output/GIS/RGBComposites')
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')
    
    composite_list = glob.glob(composites_path+'/*.tif')
    
    # The combined version is to be deployed on GEONODE
    combined_composite_filename = os.path.join(composites_path, "combined.tif")
    
    merge_rasters(composite_list, combined_composite_filename)
    
    # Then we transfer the results to MinIO
    # Processed datacubes are now stored under
    cubes_path = os.path.join(specim_mission_folder, 'processed/Output/GIS/HSIDatacubes')
    cubes_path_minio = os.path.join(specim_mission_folder_minio, 'processed/cubes')

    # Processed ancillary cubes are now stored under
    anc_cube_path = os.path.join(specim_mission_folder, 'processed/Output/GIS/AncillaryData')
    anc_cube_path_minio = os.path.join(specim_mission_folder_minio, 'processed/ancillary')

    # Processed composites are now stored under
    composites_path = os.path.join(specim_mission_folder, 'processed/Output/GIS/RGBComposites')
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')

    # Move datacube, composites and ancillary data to persistant storage on Minio
    # Cubes (radiance and reflectance)
    sb.storage.copy_folder(src_fold = cubes_path, dst_fold = cubes_path_minio, client=minio_client, containing_folder = False)
    
    # Composite data (One per transect and a combined for visualization)
    sb.storage.copy_folder(src_fold = composites_path, dst_fold = composites_path_minio, client = minio_client, containing_folder = False, overwrite = False)
    #sb.storage.copy_file(combined_composite_filename, os.path.join(composites_path_minio,'combined.tif'), client = minio_client, overwrite = True)
    
    
    # Ancillary data for further processing
    sb.storage.copy_folder(src_fold = anc_cube_path, dst_fold = anc_cube_path_minio, client = minio_client, containing_folder = False)


### 4.3 Publish data to Geonode/Geoserver

In [ ]:
# Identify datasets for publishing. Folders must contain either an ODM or Pix4D
# original orthophoto (not both) and must not contain a COG named f'{layer_name}.tif'.
# Folders must also have 'config.seabee.yaml' files where 'publish' is True
publish_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and sb.ortho.check_subdir_exists(f.parent, "processed")
    and sb.ortho.parse_config(f.parent)["publish"]
]

print("The following missions will be published to GeoNode:")
print(publish_list)

In [ ]:
import rasterio as rio
import yaml
# Publish publishables (equivalent to the same section in flights_runner.ipynb without the ODM/Pix4D stuff which does not apply here)
for mission_fold in publish_list:
    mission_name = mission_fold.name
    print(f"\n################\nProcessing: {mission_name}")
    print("Preparing orthophoto for publishing.")

    # Orthophoto is termed "combined.tif"
    ortho_path = os.path.join(
        mission_fold, "processed/composites", "combined.tif"
    )
    
    config_yaml_minio = os.path.join(
        mission_fold, "config.seabee.yaml"
    )
    
    specim_mission_folder = os.path.join(temp_dir, mission_name)
    
    h5_dir = os.path.join(specim_mission_folder, 'processed/Input/H5')
    
    nfiles = len(os.listdir(h5_dir))
    
    print(nfiles)
    
    # Prior to updating metadata we need to set the "nfiles" entry in config.seabee.yaml
    
    config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")
    
    # Add entry nfiles 
    with open(config_yaml, 'r+') as file:  
        config_data = yaml.safe_load(file)
        config_data['nfiles'] = nfiles
        # After modifying the data, write it back to the file
        yaml.safe_dump(config_data, file)
        
    with open(config_yaml, 'r') as file:  
        config_data = yaml.safe_load(file)
        print(config_data['nfiles'])
    
    # Overwrite minio config version
    sb.storage.copy_file(config_yaml, config_yaml_minio, client = minio_client, overwrite = True)
    
    
    
    # Standardise and save locally
    layer_name = sb.ortho.get_layer_name(mission_fold)
    temp_path = os.path.join(temp_dir, layer_name + ".tif")
    
    try:
        sb.geo.standardise_orthophoto(
            ortho_path,
            temp_path)
    except CalledProcessError as e:
        print(f"Failed to standardise {ortho_path}.")
        print(e)
        continue

    # Copy to MinIO and delete local version
    stan_path = os.path.join(mission_fold, "orthophoto", layer_name + ".tif")
    sb.storage.copy_file(temp_path, stan_path, minio_client, overwrite=True)
    os.remove(temp_path)

    print("Uploading to GeoServer.")

    sb.geo.upload_raster_to_geoserver(
        stan_path,
        SETTINGS.GEOSERVER_USER,
        SETTINGS.GEOSERVER_PASSWORD,
        workspace="geonode",
    )

    print("Publishing to GeoNode.")

    sb.geo.publish_to_geonode(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        workspace="geonode",
    )
    
    

    print("Updating metadata.")
    date = sb.ortho.parse_mission_data(mission_fold, parse_date=True)[2]
    abstract = sb.geo.get_html_abstract(str(mission_fold))
    metadata = {
        "abstract": abstract,
        "date": date.isoformat(),
        "date_type": "creation",
        "attribution": "SeaBee",
    }
    sb.geo.update_geonode_metadata(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        metadata,
    )

### 4.4 Delete local data

In [ ]:
"""Delete the local data. This should only be done once everything is well in place"""
for specim_mission_folder_minio in mission_list:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    shutil.rmtree(specim_mission_folder)